In [1]:
import datetime
import gc
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
# widen jupyter display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 100)
plt.style.use('seaborn')

#### in a kaggle kernel ####
# from kaggle.competitions import twosigmanews
# env = twosigmanews.make_env()
# train = env.get_training_data()
# test = []
# for (market_obs, news_obs, predictions_template) in env.get_prediction_days():
#     test.append((market_obs, news_obs, predictions_template))
#     predictions_template.confidenceValue = 0.0
#     env.predict(predictions_template)

#### markettuple is a tuple
#### ((train_mkt, train_news), [(test_mkt, test_news, predictions_template)])
#### Where train is 2008 - 2016 and test is 2017 -
# markettuple = (train, test)

#### to dump on kaggle kernel to local machine
### Data can be downloaded here
### https://www.kaggleusercontent.com/kf/8462790/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..0tg-orjoMRCx0BGA2qXftw.OSv99PzfpWWJwCEVQYq5o-i3F7kS_zkCI3G3fVRK502-Jc8n2I93M9YCym8vlh9bJP50Pp1N1BaxDVwJsr1N35hXjzWD7bv6dHRm0MGCdgNsQfP4cJIy0krnO3gvCKYZSDirp3Ux1AeSUU4eV472nw.FMlRZVHOsCsVtzo4aBPGvA/data.p
### From here
### https://www.kaggle.com/vodkahaze/data-download/output
# pickle.dump(markettuple, open('data.p', 'wb'))

#### On local machine ####
markettuple = pickle.load(open('data.p', "rb"))

In [2]:
SAMPLED_STOCKS = 850 # Sample on n stocks
NEWS_SAMPLE = 1_000_000
TEST_DAYS = 183
EARLIEST_DAY = '2009-01-01'

# NN params
BATCH_SIZE = 10000
LOOK_BACK = 30
LOOK_BACK_STEPS = 5
EPOCHS = 20
STEPS_PER_EPOCHS = 50
VALIDATION_STEPS = 5

In [3]:
mkt = markettuple[0][0].loc[
    markettuple[0][0].time >= EARLIEST_DAY].copy()
if SAMPLED_STOCKS < mkt.assetCode.nunique():
    accountsample = list(mkt.assetCode.sample(SAMPLED_STOCKS).unique())
    mkt = mkt.loc[mkt.assetCode.isin(accountsample)]
nw = markettuple[0][1].loc[
    markettuple[0][1].time >= EARLIEST_DAY].copy()
if NEWS_SAMPLE < len(nw):
    nw = nw.sample(NEWS_SAMPLE)
# news dataset has assetCodes in
#   "{'asset1', 'asset2', 'asset3'}" format
# Can select on this using a regex match
if SAMPLED_STOCKS < mkt.assetCode.nunique():
    patt = "|".join(["{}".format(i[:-2]) for i in accountsample])
    nw = nw.loc[nw.assetCodes.str.contains(patt, regex=True, case=False)]

markettuple = None
gc.collect()
mkt = mkt.sort_values(by='time'
        ).reset_index(drop=True)
nw = nw.sort_values(by='time'
      ).reset_index(drop=True)

gc.collect()

14

In [4]:
def market_prepro(
    df_mkt, 
    label_col = "returnsOpenNextMktres10",
    clean_data=True, 
    extract_time_info=True,
    scale_numeric=False,
    encode_asset=True,
    fillna_clean_outliers=True,
    ):
    """
    Preprocess market data
    """
    df = df_mkt.copy()
    # numeric columns
    ncols = ['volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 
             'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevRaw10', 
             'returnsOpenPrevRaw10', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
    if scale_numeric is True and fillna_clean_outliers is False:
        raise ValueError("scale_numeric can't be done without fillna")
    # Clean bad data. We fit on train dataset and it's ok to remove bad data
    # Remove strange cases with close/open ratio > 2
    if clean_data:
        max_ratio  = 2
        df = df[(df['close'] / df['open']).abs() <= max_ratio].loc[:]
        df = df.reset_index(drop=True)
    # Fill na, fix outliers. Safe for test dataset, no rows removed.
    if fillna_clean_outliers:
        # Fill nans
        df[ncols] = df[['assetCode'] + ncols
                                  ].groupby('assetCode'
                                  ).transform(lambda g: g.fillna(method='bfill'))
        df[ncols] = df[ncols].fillna(0)
        # Fix outliers
        df[ncols] = df[ncols].clip(
                        df[ncols].quantile(0.01), 
                        df[ncols].quantile(0.99), 
                        axis=1)
    # Extract day, week, year from time
    if extract_time_info:
        df = df.join(pd.get_dummies(
            df.time.dt.year, prefix="year",
                dummy_na=True, drop_first=True))
        df = df.join(pd.get_dummies(
            df.time.dt.day, prefix="day",
                dummy_na=True, drop_first=True))
        df = df.join(pd.get_dummies(
            df.time.dt.week, prefix="week",
                dummy_na=True, drop_first=True))
        df = df.join(pd.get_dummies(
            df.time.dt.dayofweek, prefix="dayofweek",
                dummy_na=True, drop_first=True))
        # Create linear time index column
        udays = sorted(list(df.time.unique()))
        timeindexdf = pd.DataFrame(np.arange(len(udays)), 
                                   index=udays
                                  ).reset_index(drop=False)
        timeindexdf.columns = ['time', 'timeIndex']
        df = pd.merge(df, timeindexdf, how='left', on='time')
    if encode_asset:
        df = df.join(pd.get_dummies(
                df['assetCode'], prefix="assetCode",
                dummy_na=True, drop_first=True))
    if scale_numeric:
        # Fit for numeric and time
        df[ncols] = StandardScaler().fit_transform(
                                df[ncols].astype(float))
    feature_cols = list(df.columns)
    return df[feature_cols].copy()

In [5]:
def news_prepro(
    df_news,
    scale_numeric = True,
    ):
    """
    Aggregate news by day and asset. Normalize numeric values.
    Prepare news batch for generator.
    Asset can have many news per day, so group them by asset, day and aggregate. 
    Then normalize numerical values. News aggregation part is based on this kernel: 
    https://www.kaggle.com/bguberfain/a-simple-model-using-the-market-and-news-data#
    """
    news_cols_agg = {
        'urgency': ['min', 'count'],
        'takeSequence': ['max'],
        'bodySize': ['min', 'max', 'mean', 'std'],
        'wordCount': ['min', 'max', 'mean', 'std'],
        'sentenceCount': ['min', 'max', 'mean', 'std'],
        'companyCount': ['min', 'max', 'mean', 'std'],
        'marketCommentary': ['min', 'max', 'mean', 'std'],
        'relevance': ['min', 'max', 'mean', 'std'],
        'sentimentNegative': ['min', 'max', 'mean', 'std'],
        'sentimentNeutral': ['min', 'max', 'mean', 'std'],
        'sentimentPositive': ['min', 'max', 'mean', 'std'],
        'sentimentWordCount': ['min', 'max', 'mean', 'std'],
        'noveltyCount12H': ['min', 'max', 'mean', 'std'],
        'noveltyCount24H': ['min', 'max', 'mean', 'std'],
        'noveltyCount3D': ['min', 'max', 'mean', 'std'],
        'noveltyCount5D': ['min', 'max', 'mean', 'std'],
        'noveltyCount7D': ['min', 'max', 'mean', 'std'],
        'volumeCounts12H': ['min', 'max', 'mean', 'std'],
        'volumeCounts24H': ['min', 'max', 'mean', 'std'],
        'volumeCounts3D': ['min', 'max', 'mean', 'std'],
        'volumeCounts5D': ['min', 'max', 'mean', 'std'],
        'volumeCounts7D': ['min', 'max', 'mean', 'std'],
    }
    news_cols_numeric = set(news_cols_agg.keys()) - set(['assetCode', 'time'])
    # Fill na with previous value for the asset
    df = df_news.copy()
    # Aggregate by time, asset code
    # Fix asset codes (str -> list)
    # Since asset codes are in {1, 2, 3} format
    # We need to repeat rows for each asset code
    df['assetCodes'] = df['assetCodes'].str.findall(f"'([\w\./]+)'")
    # Group to date level
    df.time = df.time.astype('datetime64[D, UTC]')
    # Expand assetCodes
    assetCodes_expanded = list(itertools.chain(*df['assetCodes']))
    if not df.empty: 
        assetCodes_index = df.index.repeat(df['assetCodes'].apply(len)) 
    else: assetCodes_index = df.index
    assert len(assetCodes_index) == len(assetCodes_expanded)
    df_assetCodes = pd.DataFrame({'level_0': assetCodes_index, 
                                  'assetCode': assetCodes_expanded})
    # Create expanded news (will repeat every assetCodes' row)
    news_cols = ['time', 'assetCodes'] + sorted(list(news_cols_agg.keys()))
    df_expanded = pd.merge(df_assetCodes, df[news_cols],
                           left_on='level_0', right_index=True, 
                           suffixes=(['','_old']))
    # Aggregate numerical news features
    df_agg = df_expanded.groupby(['time', 'assetCode']).agg(news_cols_agg)
    # Flat columns
    df_agg.columns = ['_'.join(col).strip() for col in df_agg.columns.values]
    # Normalize, fillna etc. Don't remove rows.
    df_agg = df_agg.fillna(0)
    if not df_agg.empty:
        news_df_numeric = df_agg._get_numeric_data().astype(float)
        df_agg[news_df_numeric.columns] = StandardScaler().fit_transform(news_df_numeric)
    return df_agg

In [6]:
def joined_prepro(
    mkt_df, nw_df,
    label_col = "returnsOpenNextMktres10",
    quant_features=False,
    clean_data=False, 
    extract_time_info=True,
    scale_numeric=False,
    encode_asset=True,
    fillna_clean_outliers=True,
    time_shifts=0,
    time_shift_cols=[
        "volume", "close", "open",]
    ):
    """
    Returns X
    """
    # market has index [time, assetCode]
    dfmkt = market_prepro(mkt_df,
        label_col=label_col,
        clean_data=clean_data, 
        extract_time_info=extract_time_info,
        scale_numeric=scale_numeric,
        encode_asset=encode_asset,
        fillna_clean_outliers=fillna_clean_outliers)
    # We select news in train time interval
    dfnews = news_prepro(nw_df,
        scale_numeric=scale_numeric)
    # Join by index, which is time, assetCode. 
    # Some assets have no news at all, so left join and 0 nans
    X = dfmkt.merge(dfnews, how='left', left_on=['time', 'assetCode'], 
                    right_on=['time','assetCode'],  right_index=True)
    if fillna_clean_outliers:
        # Some market data can be without news, fill nans
        X.loc[:, 'urgency_min':] = X.loc[:, 'urgency_min':].fillna(0.0)
    # Drop non-training columns
    y = X.returnsOpenNextMktres10.copy()
    if time_shifts > 0:
        X = X.sort_values(by=['assetCode', 'time']).reset_index(drop=True)
        shifts = []
        for col_name in time_shift_cols:
            for i in range(1, time_shifts + 1):
                shifts.append(X[["assetCode", col_name]].groupby("assetCode").shift(i))
                shifts[-1].rename(lambda coln: coln + "_L" + str(i), 
                                  axis='columns',
                                  inplace=True)
        shifts = pd.concat(shifts, axis=1)
        X = X.join(shifts)
        X.dropna(axis=0, how='any').reset_index(drop=True)
        gc.collect()
    X = X.drop(['assetCode', 'assetName', label_col], axis=1)
    gc.collect()
    X = X.fillna(0.0)
    gc.collect()
    X = X.reset_index(drop=True)
    gc.collect()
    return X, y

# Flat Autoregressive Model

In [7]:
X, y = joined_prepro(
    mkt, nw,
    label_col = "returnsOpenNextMktres10",
    quant_features=False,
    clean_data=False,
    extract_time_info=True,
    scale_numeric=False,
    encode_asset=True,
    fillna_clean_outliers=True,
    time_shifts=48,
    time_shift_cols=[
        "volume",
        "close", 
        "open",
        "returnsClosePrevRaw1", 
        "returnsOpenPrevRaw1", 
        "returnsClosePrevMktres1", 
        "returnsOpenPrevMktres1", 
        "returnsClosePrevRaw10",
        "returnsOpenPrevRaw10",
        "returnsClosePrevMktres10", 
        "returnsOpenPrevMktres10",
    ])

latest_day = X.time.max()
test_start = latest_day - datetime.timedelta(days=TEST_DAYS)

X_test = X.loc[X.time >= test_start].copy()
X = X.loc[X.time < test_start]
gc.collect()
y_test = y.loc[~y.index.isin(X.index)].copy()
y = y.loc[y.index.isin(X.index)]
gc.collect()

mkt = None
nw = None
gc.collect()

X = X.drop(['time'], axis=1)

# The index of the columns of the asset codes
# used to generate batches by asset codes
asset_code_column_indices = [
    i for i, x in enumerate(X.columns) 
    if "assetcode_" in x.lower()
]

X = X.values
y = y.values * 100
X_test = X_test.values
y_test = y_test.values * 100
gc.collect()

7

In [8]:
def generate_by_stock(X, y, asset_idx_list, batch_size):
    """
    Generate batches by sampling from individual stocks 
    
    NOTE:
        BATCH SIZE IS CURRENTLY UNUSED. could be used to sample from the samples
    """
    while True:
        # random asset
        _asset = np.random.choice(asset_idx_list)
        # Get row idx for where the asset one hot encoded feature is true 
        _loc = X[:, _asset] > 0
        batch_features = X[_loc]
        batch_labels = y[_loc]
        yield batch_features, batch_labels

In [9]:
from keras import backend as K
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LSTM, Embedding
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasRegressor

def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true-y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - (SS_res / (SS_tot + K.epsilon()))

earlystopper = EarlyStopping(monitor='mean_squared_error', patience=5, verbose=1)

model = Sequential()
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', 
              optimizer='adam', 
              metrics=[r2_score])

Using TensorFlow backend.


In [12]:
model.fit(X, y,
   epochs=25,
   batch_size=32, 
   verbose=1,
   callbacks=[earlystopper])

# model.fit_generator(
#     generator=generate_by_stock(
#         X, y, 
#         asset_idx_list=asset_code_column_indices, 
#         batch_size=BATCH_SIZE),
#     steps_per_epoch=len(asset_code_column_indices), 
#     epochs=10,
#     verbose=1, 
#     callbacks=[earlystopper]
# )

Epoch 1/10
712/712 [==============================] - 36s 51ms/step - loss: nan - r2_score: nan
Epoch 2/10
  3/712 [..............................] - ETA: 33s - loss: nan - r2_score: nan

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `mean_squared_error` which is not available. Available metrics are: loss,r2_score
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


712/712 [==============================] - 34s 48ms/step - loss: nan - r2_score: nan
Epoch 3/10
712/712 [==============================] - 32s 45ms/step - loss: nan - r2_score: nan
Epoch 4/10
  8/712 [..............................] - ETA: 29s - loss: nan - r2_score: nan

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)

# LSTM Model

In [ ]:
raise Exception("Just making sure you don't train the LSTM needlessly")

In [ ]:
class LSTMGenerator:
    """
    Keras standard approach to generage batches for model.fit_generator() call.
    """
    def __init__(self, prepro, market, news, index_df):
        """
        @param preprocessor: market and news join preprocessor
        @param market: full loaded market df
        @param news: full loaded news df
        @param index_df: df with assetCode and time of train or validation market data. Batches will be taken from them.
        """
        self.market = market
        self.prepro = prepro
        self.news = news
        self.index_df = index_df
        self.asset_codes = self.index_df['assetCode'].unique().tolist()

    def flow_lstm(self, batch_size, is_train, look_back, look_back_step):
        """
        Generates batch data for LSTM NN
        Each cycle in a loop we yield a batch for one training step in epoch. 
        """
        while True:
            # Get market indices of random assets, sorted by assetCode, time.
            batch_index_df = self.get_random_assets_idx(batch_size)
            # Get X, y data for this batch, containing market and news, but without look back yet
            X, y = self.get_batch(batch_index_df, is_train)
            # Add look back data to X, y
            X, y = self.with_look_back(X,y,look_back,look_back_step)
            yield X,y
    
    def get_random_assets_idx(self, batch_size):
        """
        Get random asset and it's last market data indices.
        Repeat for next asset until we reach batch_size.
        """
        # Insert first asset
        asset = np.random.choice(self.asset_codes)
        asset_codes.remove(asset)
        batch_index_df = self.index_df[self.index_df.assetCode == asset].tail(batch_size)
        # Repeat until reach batch_size records
        while (batch_index_df.index.size < batch_size) and (len(asset_codes) > 0):
            asset = np.random.choice(asset_codes)
            asset_codes.remove(asset)
            asset_index_df = self.index_df[self.index_df.assetCode == asset].tail(batch_size - batch_index_df.index.size)
            batch_index_df = pd.concat([batch_index_df, asset_index_df])
        
        return batch_index_df.sort_values(by=['assetCode', 'time'])
            
    def get_batch(self, batch_idx, is_train):
        """
        Get batch of market-news data without lookback
        """
        market_df = self.market.loc[batch_idx.index]
        # Select subset of news for future merge by assetCode and time. 
        news_df = self.news.merge(batch_idx, on=['time'])
        # Remove bad rows, clean the data. It's ok for train.
        if is_train: 
            market_df, news_df = prepro.fix_train(market_df, news_df)
        # Join market and news using preprocessor       
        X = self.prepro.get_X(market_df, news_df)
        y = self.prepro.get_y(market_df)
        return(X, y)
    
    # convert an array of values into a dataset matrix
    def with_look_back(self, X, y, look_back, look_back_step):
        """
        Add look back window values to prepare dataset for LSTM
        """
        X_processed, y_processed = [], []
        # Fix last window in batch, can be not full
        if look_back > len(X): 
            look_back = len(X)
            look_back_step = min(look_back_step, look_back)
            
        for i in range(0,len(X)-look_back+1):
            # Add lookback to X
            x_window = X.values[i:(i+look_back):look_back_step, :]
            X_processed.append(x_window)
            # If input is X only, we'll not output y
            if y is None: continue
            # Add lookback to y
            y_window = y.values[i+look_back-1, :]
            y_processed.append(y_window)
        # Return Xy for train/test or X for prediction
        if(y is not None): return np.array(X_processed), np.array(y_processed)
        else: return np.array(X_processed)

    
# Train data generator instance
join_generator = LSTMGenerator(prepro, mkt, nw, mkt)

# Validation data generator instance
val_generator = LSTMGenerator(prepro, mkt, nw, mkt)

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization, LSTM, Embedding
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical

model = Sequential()
# Add an input layer market + news
input_size = len(market_prepro.feature_cols) + len(news_prepro.feature_cols)
# input_shape=(timesteps, input features)
model.add(LSTM(units=128, return_sequences=True, input_shape=(None,input_size)))
model.add(LSTM(units=64, return_sequences=True ))
model.add(LSTM(units=32, return_sequences=False))
# Add an output layer 
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
weights_file='best_weights.h5'
# We'll stop training if no improvement after some epochs
earlystopper = EarlyStopping(patience=5, verbose=1)
# Low, avg and high scor training will be saved here
# Save the best model during the traning
checkpointer = ModelCheckpoint(
    weights_file,
    verbose=1,
    save_best_only=True,
    save_weights_only=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.001)

training = model.fit_generator(
    join_generator.flow_lstm(
        batch_size=BATCH_SIZE,
        is_train=True, 
        look_back=LOOK_BACK, 
        look_back_step=LOOK_BACK_STEPS),
        epochs=EPOCHS,
        validation_data=val_generator.flow_lstm(
            batch_size=BATCH_SIZE,
            is_train=False, 
            look_back=LOOK_BACK, 
            look_back_step=LOOK_BACK_STEPS),
    steps_per_epoch=STEPS_PER_EPOCHS,
    validation_steps=VALIDATION_STEPS,
    callbacks=[earlystopper, checkpointer, reduce_lr])